<a href="https://colab.research.google.com/github/rishank012/Capstone-Project/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import pathway as pw
import bokeh.plotting
from bokeh.models import LinearAxis, Range1d, HoverTool
from bokeh.transform import linear_cmap
import panel as pn
import os

# Step 1: Importing and Preprocessing the Data

In [3]:
    from google.colab import files
    uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [16]:
df = pd.read_csv('dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [5]:
to_num = ['ID','Occupancy','Capacity','QueueLength','TrafficConditionNearby','IsSpecialDay']
df[to_num] = (
    df[to_num]
      .apply(lambda c: pd.to_numeric(c, errors='coerce'))
      .fillna(0)
)

In [6]:
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)
df = df.sort_values(['ID','Timestamp']).reset_index(drop=True)

# Feature Engineering:

In [7]:
df['occ_rate'] = df['Occupancy'] / df['Capacity']

for col in ['QueueLength','TrafficConditionNearby']:
    mn = df.groupby('ID')[col].transform('min')
    mx = df.groupby('ID')[col].transform('max')
    df[f'{col}_norm'] = (df[col] - mn) / (mx - mn + 1e-6)

vt_map = {'car':1.0,'bike':0.5,'truck':1.5}
df['VT_weight'] = df['VehicleType'].map(vt_map).fillna(1.0)

### **Model 1:** *Baseline Linear Model*

In [8]:
alpha = 0.5
df['delta1'] = alpha * df['occ_rate']
df['price_m1'] = df.groupby('ID')['delta1'].cumsum().add(10)

### **Model 2:** *Demand Based Price Function*

In [9]:
a,b,c,d,e = 1.0,0.8,0.6,1.2,0.5
lam = 0.8

df['raw_d'] = (
      a*df['occ_rate']
    + b*df['QueueLength_norm']
    - c*df['TrafficConditionNearby_norm']
    + d*df['IsSpecialDay']
    + e*df['VT_weight']
)
dmin = df.groupby('ID')['raw_d'].transform('min')
dmax = df.groupby('ID')['raw_d'].transform('max')
df['d_norm'] = (df['raw_d'] - dmin) / (dmax - dmin + 1e-6)

df['price_m2'] = 10*(1 + lam*df['d_norm'])
df['price_m2'] = df['price_m2'].clip(5,20)

###**Model 3:** *Competitive Pricing Model*

In [10]:
def comp_logic(r):
    if r['Occupancy'] >= r['Capacity']:
        r['price_m3'] = r['price_m2'] * 0.7
        r['suggest_reroute'] = True
    else:
        uplift = 0.1 * r['price_m2'] * (1 - r['TrafficConditionNearby_norm'])
        r['price_m3'] = r['price_m2'] + uplift
        r['suggest_reroute'] = False
    return r

df = df.apply(comp_logic, axis=1)

In [11]:
for m in ['price_m1','price_m2','price_m3']:
    df[f'{m}_sm'] = df.groupby('ID')[m].transform(lambda x: x.ewm(alpha=0.2).mean())

In [12]:
out = [
  'Timestamp','ID','Occupancy','Capacity','occ_rate',
  'QueueLength_norm','TrafficConditionNearby_norm','IsSpecialDay',
  'VT_weight','price_m1','price_m2','price_m3',
  'price_m1_sm','price_m2_sm','price_m3_sm','suggest_reroute'
]
df[out].to_csv('parking_stream_final.csv', index=False)

### **Defining Pathway Schema:**

In [13]:
class ParkingSchema(pw.Schema):
    Timestamp: str; ID: int; Occupancy: int; Capacity: int
    occ_rate: float; QueueLength_norm: float; TrafficConditionNearby_norm: float
    IsSpecialDay: int; VT_weight: float
    price_m1: float; price_m2: float; price_m3: float
    price_m1_sm: float; price_m2_sm: float; price_m3_sm: float
    suggest_reroute: bool

stream = pw.demo.replay_csv(
    'parking_stream_final.csv',
    schema=ParkingSchema,
    input_rate=500
).with_columns(
    t = pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)

## **Bokeh visualization of all three models:**

In [17]:
def enhanced_plot(src):
    fig = bokeh.plotting.figure(
        height=450, width=950,
        title="Dynamic Pricing Models w/ Occupancy Overlay",
        x_axis_type="datetime"
    )
    # Smooth prices
    fig.line('t','price_m1_sm', source=src, color='blue',  line_width=2, legend_label="Model 1 (EWMA)")
    fig.line('t','price_m2_sm', source=src, color='green', line_width=2, legend_label="Model 2 (EWMA)")
    fig.line('t','price_m3_sm', source=src, color='red',   line_width=2, legend_label="Model 3 (EWMA)")

    # Occupancy on right axis
    fig.extra_y_ranges = {"occ": Range1d(0,1)}
    fig.add_layout(LinearAxis(y_range_name="occ", axis_label="Occupancy Rate"), 'right')
    fig.line('t','occ_rate', source=src, y_range_name="occ",
             color='orange', line_dash='dotted', line_width=2,
             legend_label="Occupancy Rate")

    # Reroute highlight via color map (gray=false, black=true)
    cmap = linear_cmap('suggest_reroute', ['lightgray','black'], 0,1)
    fig.circle('t','price_m3', source=src, size=6, color=cmap, legend_label="Reroute Flag")

    # Hover
    hover = HoverTool(tooltips=[
        ("Time", "@t{%F %H:%M}"),
        ("Model 1", "@price_m1{0.00}"),
        ("Model 2", "@price_m2{0.00}"),
        ("Model 3", "@price_m3{0.00}"),
        ("Occ", "@occ_rate{0.0%}")
    ], formatters={'@t':'datetime'}, mode='vline')
    fig.add_tools(hover)

    fig.legend.location = "top_left"
    fig.legend.click_policy = "hide"
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price (USD)"

    return fig

pn.extension()
viz = stream.plot(enhanced_plot, sorting_col='t')
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [15]:
os.environ['PW_RUN_SANDBOX'] = 'true'
pw.run()

Output()